In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
from wordcloud import WordCloud
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

import os
import sys
from dotenv import load_dotenv

load_dotenv()
REPO_PATH =  os.getenv('REPO_PATH')
plt.style.use('science')

sys.path.insert(0, rf'{REPO_PATH}src_HF')
from utils.main_utils import *
from utils.text_utils import *

### Import data

In [ ]:
TOPIC = 'CRU'

text_df = pd.read_json(rf'{REPO_PATH}data\news_data\EIKON_{TOPIC}_NEWS_COMPLETE.json', lines=True, orient='records')

text_df['tokenized'], text_df['tokenized_cleaned'] = clean_token_series(text_df['fullStory'])

display(text_df)

### Create a word dataframe

In [ ]:
word_series = text_df['tokenized_cleaned'].explode()
word_series = word_series[~word_series.isin(IGNORE_WORDS)]

word_df = pd.DataFrame(word_series.value_counts()).reset_index().rename(columns={'index': 'word', 'tokenized_cleaned': 'count'})
sia = SentimentIntensityAnalyzer()
word_df['sia'] = word_df['word'].apply(lambda x: sia.polarity_scores(x)['compound'])

word_df['textblob'] = word_df['word'].apply(lambda x: TextBlob(x).sentiment.polarity)
display(word_df)

In [ ]:
sentiment_id= 'sia'

conditions = [
    word_df[sentiment_id] < -0.5,
    (word_df[sentiment_id] < -0.15) & (word_df[sentiment_id] > -0.5),
    (word_df[sentiment_id] > 0.15) & (word_df[sentiment_id] < 0.5),
    word_df[sentiment_id] > 0.5,
]

condition_names = [
    'Negative sentiment',
    'Slightly Negative sentiment',
    'Slightly Positive sentiment',
    'Positive sentiment'
]

fig, axs = plt.subplots(2, 2, facecolor=None, figsize=(12, 12))

for i, ax in enumerate(axs.flatten()):

    filtered_word_series = word_df[conditions[i]]['word']
    print(len(filtered_word_series))
    wordcloud = create_wordcloud(filtered_word_series)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.set_title(f'{condition_names[i]}', fontsize=19, loc='left', y=1.05)
    ax.axis("off")


fig.tight_layout(pad=4)

fig.savefig(rf'{REPO_PATH}src_HF\2 Data Cleaning and EDA\images\sentiment_wordcloud.png')

In [ ]:
word_df['word_count'] = 1

# sum word_count where both sia and textblob are the same
word_df_grouped = word_df.groupby(['sia', 'textblob']).sum().reset_index()

# remove row where both values are 0
word_df_grouped = word_df_grouped[(word_df_grouped['sia'] != 0) | (word_df_grouped['textblob'] != 0)]

word_df_grouped
fig = word_df_grouped.plot.scatter(
    x='sia', 
    y='textblob', 
    figsize=(10, 7), 
    c='word_count', 
    s='word_count', 
    colormap='viridis'
)

# increase label sizes
fig.set_xlabel('SIA', fontsize=15)
fig.set_ylabel('TextBlob', fontsize=15)
fig.tick_params(axis='both', which='major', labelsize=12)
fig.grid(alpha=0.3)


cbar = fig.collections[0].colorbar
cbar.ax.tick_params(labelsize=12)
cbar.set_label('Word count', fontsize=15)

fig.figure.savefig(repo_path + r'src_HF\2 Data Cleaning and EDA\images\sentiment_comparison.png')

### Headline and story length distribution

In [ ]:
text_df['tokenized_h'], text_df['tokenized_cleaned_h'] = clean_token_series(text_df['text'])

# remove punctuation from tokenized_h and tokenized
text_df['tokenized_h_words'] = text_df['tokenized_h'].apply(lambda x: [word for word in x if word.isalnum()])
text_df['tokenized_words'] = text_df['tokenized'].apply(lambda x: [word for word in x if word.isalnum()])

In [ ]:
word_counts_headline = text_df['tokenized_h_words'].apply(len)
word_counts_story = text_df['tokenized_words'].apply(len)

# set limits
word_counts_headline = word_counts_headline[word_counts_headline < 80]
word_counts_story = word_counts_story[word_counts_story < 1500]

wc_df = pd.DataFrame({
    'headline': word_counts_headline,
    'story': word_counts_story
})

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=200)

colormap = plt.cm.get_cmap('twilight', 256)
palette = [colormap(i) for i in (70, 180)]

wc_df['headline'].plot(
    kind='hist', 
    bins=75, 
    color=palette[0],
    edgecolor='black',
    ax=ax1,
    )

wc_df['story'].plot(
    kind='hist', 
    bins=75, 
    color=palette[1],
    edgecolor='black',
    ax=ax2,
    )

for ax, label in zip((ax1, ax2), ('Headline ', 'Full text ')):
    ax.set_xlabel(label + 'word count', fontsize=13)
    ax.set_ylabel('Frequency', fontsize=13)
    ax.grid(alpha=0.3)
    ax.set_axisbelow(True)

fig.figure.savefig(f'images/news_word_count_dist.png', bbox_inches='tight')